In [2]:
import torch
import torch.nn as nn
from torchdiffeq import odeint

In [12]:
from scipy.integrate import solve_ivp
import numpy as np
import pandas as pd

def lorenz(t, state, sigma=10, rho=28, beta=8/3):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

In [13]:
def generate_lorenz_data(state=[1.0, 1.0, 1.0], length=100, points=1000):
    """
    Generates data for the Lorenz system.

    Parameters:
        state (list): Initial state [x, y, z].
        length (float): Length of the time interval.
        points (int): Number of points to evaluate.

    Returns:
        pd.DataFrame: A DataFrame with Lorenz system data.
    """
    t_span = (0, length)
    t_eval = np.linspace(*t_span, points)
    solution = solve_ivp(lorenz, t_span, state, t_eval=t_eval)
    
    df = pd.DataFrame(solution.y.T, columns=["X", "Y", "Z"])
    return df

In [5]:
class ODEFunc(nn.Module):
    def __init__(self, hidden_dim):
        super(ODEFunc, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(hidden_dim, 50),
            nn.ReLU(),
            nn.Linear(50, hidden_dim)
        )

    def forward(self, t, h):
        return self.net(h)


In [6]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Encoder, self).__init__()
        self.rnn = nn.GRU(input_dim, hidden_dim, batch_first=True)

    def forward(self, x):
        """
        x: [batch_size, seq_length, input_dim] (multivariate time series)
        Returns:
        h0: [batch_size, hidden_dim] (hidden state representation)
        """
        _, h0 = self.rnn(x)
        return h0.squeeze(0)


In [9]:
class Decoder(nn.Module):
    def __init__(self, hidden_dim, output_dim):
        super(Decoder, self).__init__()
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, h):
        """
        h: Hidden state [batch_size, hidden_dim]
        Returns:
        y: Output (predicted multivariate values) [batch_size, output_dim]
        """
        return self.fc(h)


In [8]:
class ODEFunc(nn.Module):
    def __init__(self, hidden_dim):
        super(ODEFunc, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(hidden_dim, 50),  # Increase complexity for multivariate dynamics
            nn.Tanh(),
            nn.Linear(50, hidden_dim)  # Output matches hidden_dim
        )

    def forward(self, t, h):
        """
        t: Time (required by odeint but can be ignored if dynamics are time-invariant)
        h: Hidden state [batch_size, hidden_dim]
        Returns:
        dh/dt: Time derivative of the hidden state
        """
        return self.net(h)


In [10]:
class NeuralODEForecasting(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NeuralODEForecasting, self).__init__()
        self.encoder = Encoder(input_dim, hidden_dim)
        self.odefunc = ODEFunc(hidden_dim)
        self.decoder = Decoder(hidden_dim, output_dim)

    def forward(self, x, t_future):
        """
        x: Observed time-series data [batch_size, seq_length, input_dim]
        t_future: Future time points [num_time_points]
        Returns:
        y_future: Predicted values [batch_size, num_time_points, output_dim]
        """
        # Encode observed time-series into a hidden state
        h0 = self.encoder(x)

        # Predict the future hidden states using the Neural ODE
        h_future = odeint(self.odefunc, h0, t_future)

        # Decode the future hidden states to predictions
        y_future = torch.stack([self.decoder(h) for h in h_future], dim=1)
        return y_future


In [25]:
# Generate Lorenz data
data = generate_lorenz_data(state=[1.0, 1.0, 1.0], length=100, points=50000)

# Add a time column
data["Time"] = np.linspace(0, 100, 50000)
print(data)
print(len(data))


              X         Y          Z     Time
0      1.000000  1.000000   1.000000    0.000
1      1.000511  1.051968   0.996728    0.002
2      1.002034  1.103904   0.993578    0.004
3      1.004550  1.155857   0.990554    0.006
4      1.008045  1.207876   0.987657    0.008
...         ...       ...        ...      ...
49995 -0.482363 -0.827958  11.186281   99.992
49996 -0.489360 -0.842647  11.127588   99.994
49997 -0.496510 -0.857605  11.069234   99.996
49998 -0.503814 -0.872837  11.011217   99.998
49999 -0.511275 -0.888350  10.953538  100.000

[50000 rows x 4 columns]
50000


In [26]:
from sklearn.preprocessing import MinMaxScaler

# Normalize the data (X, Y, Z)
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[["X", "Y", "Z"]])
scaled_df = pd.DataFrame(scaled_data, columns=["X", "Y", "Z"])

# Create sequences for supervised learning
def create_sequences(data, sequence_length=50):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length, :])  # Historical data
        y.append(data[i+sequence_length, :])   # Next point
    return np.array(X), np.array(y)

# Create sequences
sequence_length = 10
X, y = create_sequences(scaled_data, sequence_length)

# Split into training and test sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


In [27]:
input_dim = 3  # Multivariate input (X, Y, Z)
hidden_dim = 32
output_dim = 3
model = NeuralODEForecasting(input_dim, hidden_dim, output_dim)

In [28]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Training loop
epochs = 100
t_future = torch.tensor([1.0])
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    # Convert data to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)


    
    # Forward pass
    y_pred = model(X_train_tensor, t_future)


    print("y_pred shape:", y_pred.shape)
    print("y_train_tensor shape:", y_train_tensor.shape)
    
    # Compute loss
    loss = criterion(y_pred, y_train_tensor)
    
    # Backward pass
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

y_pred shape: torch.Size([39992, 1, 3])
y_train_tensor shape: torch.Size([39992, 3])


c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([39992, 3])) that is different to the input size (torch.Size([39992, 1, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 19192320768 bytes.